In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, GRU, Bidirectional, Input

data = {
    "Hours": [0.5, 1.7, 2.5, 3.0, 4.8, 5.5, 6.3, 7.8, 8.6, 9.9],
    "Scores": [10, 21, 32, 37, 52, 61, 72, 80, 88, 98]
}

df = pd.DataFrame(data)
print("Sample Data:\n", df.head())

X = df[['Hours']].values
y = df['Scores'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

# 💻 ML Models
models = {
    "Linear Regression": LinearRegression(),
    "SVM": SVR(kernel='rbf', C=1.2, epsilon=0.2),
    "Random Forest": RandomForestRegressor(n_estimators=120, max_depth=5, random_state=42),
    "XGBoost": XGBRegressor(objective='reg:squarederror', n_estimators=120, learning_rate=0.12, random_state=42),
    "KNN": KNeighborsRegressor(n_neighbors=3),
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    results[name] = {"MSE": mean_squared_error(y_test, preds), "R2": r2_score(y_test, preds)}

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
nb = GaussianNB()
y_train_int = np.round(y_train)
nb.fit(X_train_scaled, y_train_int)
preds = nb.predict(X_test_scaled)
results["Naive Bayes (approx)"] = {"MSE": mean_squared_error(y_test, preds), "R2": r2_score(y_test, preds)}

# 🧠 MLP Regressor
mlp = MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=2000, random_state=42)
mlp.fit(X_train, y_train)
preds = mlp.predict(X_test)
results["MLP (Sklearn)"] = {"MSE": mean_squared_error(y_test, preds), "R2": r2_score(y_test, preds)}

X_train_rnn = np.expand_dims(X_train, axis=1)
X_test_rnn = np.expand_dims(X_test, axis=1)

def build_and_train(model, name):
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train_rnn, y_train, epochs=200, verbose=0)
    preds = model.predict(X_test_rnn).flatten()
    results[name] = {"MSE": mean_squared_error(y_test, preds), "R2": r2_score(y_test, preds)}

rnn = Sequential([Input(shape=(1,1)), SimpleRNN(32), Dense(1)])
build_and_train(rnn, "RNN")

lstm = Sequential([Input(shape=(1,1)), LSTM(32), Dense(1)])
build_and_train(lstm, "LSTM")

bilstm = Sequential([Input(shape=(1,1)), Bidirectional(LSTM(32)), Dense(1)])
build_and_train(bilstm, "BiLSTM")

gru = Sequential([Input(shape=(1,1)), GRU(32), Dense(1)])
build_and_train(gru, "GRU")

bigru = Sequential([Input(shape=(1,1)), Bidirectional(GRU(32)), Dense(1)])
build_and_train(bigru, "BiGRU")

print("\nModel Performance (MSE & R2):")
print(pd.DataFrame(results).T)

Sample Data:
    Hours  Scores
0    0.5      10
1    1.7      21
2    2.5      32
3    3.0      37
4    4.8      52
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step

Model Performance (MSE & R2):
                              MSE        R2
Linear Regression        3.599365  0.995249
SVM                    650.768424  0.140963
Random Forest           22.422292  0.970402
XGBoost                 88.494392  0.883184
KNN                     33.074074  0.956341
Naive Bayes (approx)    88.666667  0.882957
MLP (Sklearn)            6.714924  0.991136
RNN                   3013.157227 -2.977473
LSTM                  3383.579346 -3.466444
BiLSTM                2797.006592 -2.692147
GRU                   3182.375244 -3.200847
BiGRU                 2522.755127 -2.330126
